### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [83]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, n_iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
        ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
        latent_features - (int) the number of latent features used
        learning_rate - (float) the learning rate 
        n_iters - (int) the number of iterations
    
    OUTPUT:
        user_mat - (numpy array) a user by latent feature matrix
        movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = ratings_mat.size - np.isnan(ratings_mat).sum() # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for itr in range(n_iters):
        # initialize Mean-squared-error
        mserr = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if(ratings_mat[i, j] > 0): 
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i,:], movie_mat[:, j])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    mserr += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k]  += learning_rate * (2 * movie_mat[k, j] * diff) # Notice the index order
                        movie_mat[k, j] += learning_rate * (2 * user_mat[i, k] * diff) # Notice the decent direction
 
        # print results for iteration 
        if((itr+1) == 1):
            print('%d \t\t %f' %(itr+1, mserr/num_ratings))
        elif((itr+1)%50 == 0):
            print('%d \t\t %f' %(itr+1, mserr/num_ratings))
        elif((itr+1) == n_iters):
            print('%d \t\t %f' %(itr+1, mserr/num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [75]:
user_mat, movie_mat = FunkSVD(ratings_mat, learning_rate=0.002, n_iters=500) 

Optimization Statistics
Iterations | Mean Squared Error 
1 		 51.359033
50 		 1.352997
100 		 0.420102
150 		 0.197443
200 		 0.027792
250 		 0.002616
300 		 0.000303
350 		 0.000037
400 		 0.000005
450 		 0.000001
500 		 0.000000


In [65]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99989189   9.99995996   9.99999146  10.00012562]
 [  9.99931424   3.99975814   8.99994701  10.00078672]
 [  7.9998395    8.99994315   9.99998858   5.00018516]
 [  8.99975541   7.99991697   9.99998327  10.00027991]
 [ 10.00095462   5.00033688   9.00007627   8.99890781]
 [  6.00040502   4.00014121  10.00003146   5.99953733]
 [  9.0004588    8.00015795  10.00003512   8.99947677]
 [  9.9997079    4.99989562   8.99997615   8.00033479]
 [  7.00017691   8.00006403  10.00001345   7.99979512]
 [  9.00061473   5.00021717   9.00004695   6.9992938 ]
 [  8.99999824   8.00000006   9.99999819   7.99999925]
 [  9.00003286  10.00000915  10.00000069   8.9999622 ]
 [  9.99921974   8.99972396   9.99993766   8.0008933 ]
 [  5.00092467   8.0003248    5.00007181   7.99894079]
 [  9.99996589   7.99998613   9.9999957   10.00003868]
 [  8.99967039   8.99988026   9.99997265  10.00037892]
 [  8.99913674   7.99970103   7.99993331   8.00098485]
 [ 10.00015641   8.00005593   1.00001304   9.99982133]
 [  4.9994

The predicted rating from the dot product are already starting to look a lot like the original data values afrer only 10 iterations. 

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`3.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [84]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [86]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, n_iters=250) 

Optimization Statistics
Iterations | Mean Squared Error 
1 		 44.258797
50 		 0.119201
100 		 0.000154
150 		 0.000000
200 		 0.000000
250 		 0.000000


In [87]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.842176160336237:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`4.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [88]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, n_iters=20) 

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.640420
20 		 1.065694


`5.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [89]:
# Replace each of the comments below with the correct values
num_ratings = first_1000_users.size - np.isnan(first_1000_users).sum() # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = np.isnan(first_1000_users).sum() # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [90]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
